# Projecte Node21: Classificació i Detecció de Nòduls Pulmonars en Radiografies de Pit.

## Introducció
En aquest projecte abordarem la detecció de nòduls pulmonars en radiografies de tòrax (CXR) seguint l'enunciat del repte **Node21** ([enllaç al repte](https://node21.grand-challenge.org/)). El repte Node21 proporciona un conjunt de dades públic de radiografies frontals de pit amb nòduls pulmonars anotats mitjançant **caixes delimitadores** (bounding boxes). En total consta d'aproximadament **4.882 radiografies**, de les quals **1.134** contenen almenys un nòdul i **3.748** imatges no en contenen, actuant com a casos negatius.  

L'objectiu consisteix en aplicar tècniques d'aprenentatge automàtic al problema proposat. S'han definit un **conjunt de tasques seqüencials** per tal de facilitar-ne el desenvolupament:

1. **Classificació**: Desenvolupar un sistema de classificació binària capaç de determinar si una radiografia conté nòduls pulmonars. Aquesta tasca es resoldrà mitjançant l'avaluació de **quatre models diferents**, alguns d'entrenats des de zero i d'altres aprofitant tècniques de *transfer learning*.
2. **Detecció**: Implementar models de detecció per localitzar els nòduls en la imatge, identificant-ne la posició amb caixes delimitadores. Per aquesta tasca es faran servir **dues arquitectures diferents** basades en *deep learning*.
3. **Innovació**: S’obre la possibilitat d’aplicar tècniques avançades o enfocaments propis que ampliïn o millorin les metodologies aplicades.

Per abordar aquestes tasques, utilitzarem un únic *notebook* Jupyter que integrarà tot el codi i explicacions necessàries. S'empraran diverses tècniques i models d'**aprenentatge automàtic** i **aprenentatge profund** vists a classe.


## Preparació de l'entorn i dependències
Abans de començar, assegurem-nos de tenir instal·lades totes les **dependències** necessàries. El projecte requerirà les biblioteqües següents:

- **NumPy** - per a manipulació de dades numèriques i de taules (anotacions).
- **PyTorch** - per construir i entrenar els models de CNN i Transfer Learning.
- **Matplotlib** - per a la visualizació de gràfiques i imatges.

Podem instal·lar els paquets que falten directament des del notebool. Per exemple:

`pip install torch torchvision scikit-learn matplotlib numpy`

També assegurarem una estructura de carpetes correcta en el directori de treball actual:

- `data/` - Contindrà les dades del NODE21 (imatges i anotacions).
- `utils/` - Codi de suport reutilitzable, com ara funcions d’augmentació, classes Dataset, funcions per dibuixar caixes, etc.
- `outputs/` - Elements de sortida.
- 

## Obtenció i preparació de les dades Node21

Per dur a terme les tasques de classificació i detecció de nòduls pulmonars, utilitzarem les dades proporcionades pel repte NODE21, descarregades des del repositori oficial de Zenodo. Aquest conjunt inclou radiografies de tòrax simulades amb nòduls inserits artificialment i anotacions detallades sobre la seva posició.

A causa de la mida del conjunt complet (~35 GB), no resulta eficient entrenar directament amb totes les imatges en un entorn personal. Per això, optarem per treballar amb un subconjunt configurable del total d’imatges, mantinguent una distribució equilibrada entre imatges positives (amb nòduls) i negatives (sense nòduls), que és fonamental per garantir una bona generalització del model.

Les imatges es troben dins la carpeta:

`data/cxr_images/processed_data/images/` 

i tenen format **.mha**. Les anotacions associades es troben al fitxer 

`data/cxr_images/processed_data/simulated_metadata.csv`

Aquest fitxer indica, per a cada nòdul simulat, a quina imatge apareix i amb quines coordenades **(x, y, width, height)**. Les imatges que no apareixen en aquest fitxer es consideren **negatives** (sense nòduls). Aquest enfocament ens permet construir tant:

- **Etiquetes binàries** per a classificació: 1 si hi ha algun nòdul, 0 si no.
- **Bounding boxes per detecció**: una o més caixes per imatge, o cap.

In [15]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split

# Ruta a les imatges i anotacions
IMG_DIR = "data/cxr_images/proccessed_data/images"
ANNOTATION_FILE = "./data/cxr_images/proccessed_data/simulated_metadata.csv"

# Nombre total d'imatges a utilitzar (ajustable)
N_IMAGES = 500  # Exemple: 500 imatges en total (positives + negatives)
VAL_RATIO = 0.25  # 25% per a validació

# Mida d'entrada desitjada per al model (per exemple, 512x512)
IMG_SIZE = 512

In [19]:
# Carregam el CSV d'anotacions
df = pd.read_csv(ANNOTATION_FILE)

# Construïm diccionari de caixes per imatge
annotations_dict = {}
for _, row in df.iterrows():
    img_id = row['img_name'].replace('.mha', '')  # treim extensió
    box = [row['x'], row['y'], row['width'], row['height']]
    annotations_dict.setdefault(img_id, []).append(box)

# Construïm etiquetes binàries per classificació
labels_dict = {img_id: 1 for img_id in annotations_dict}  # imatges amb nòdul

# Llista completa d’imatges disponibles
all_imgs = [f.replace('.mha', '') for f in os.listdir(IMG_DIR) if f.endswith('.mha')]

# Afegim les negatives (sense anotació)
for img_id in all_imgs:
    if img_id not in annotations_dict:
        annotations_dict[img_id] = []
        labels_dict[img_id] = 0

In [20]:
# Extracció estratificada d’un subconjunt manejable
img_ids = list(labels_dict.keys())
img_labels = [labels_dict[i] for i in img_ids]

# Escollim N_IMAGES aleatòriament amb la mateixa proporció de classes

_, subsampled_ids, _, _ = train_test_split(
    img_ids, img_labels, test_size=N_IMAGES, stratify=img_labels, random_state=42)

# Partim en entrenament i validació (mantenint estratificació)
labels_subsampled = [labels_dict[i] for i in subsampled_ids]
X_train, X_val = train_test_split(
    subsampled_ids, test_size=VAL_RATIO, stratify=labels_subsampled, random_state=42)

print(f"Imatges seleccionades: {len(subsampled_ids)}")
print(f"Entrenament: {len(X_train)} — Validació: {len(X_val)}")

Imatges seleccionades: 500
Entrenament: 375 — Validació: 125


In [26]:
import SimpleITK as sitk
import cv2
import numpy as np

# Carpeta de sortida
OUTPUT_DIR = f"./data/preprocessed_{IMG_SIZE}"
os.makedirs(OUTPUT_DIR, exist_ok=True)

def convert_and_resize(img_id, output_path):
    mha_path = os.path.join(IMG_DIR, f"{img_id}.mha")
    img = sitk.ReadImage(mha_path)
    arr = sitk.GetArrayFromImage(img)[0]  # imatge 2D
    
     # Normalització
    norm = ((arr - arr.min()) / (arr.max() - arr.min())) * 255
    
    resized = cv2.resize(norm, (IMG_SIZE, IMG_SIZE))
    cv2.imwrite(os.path.join(output_path, f"{img_id}.png"), resized)

# Convertim les imatges seleccionades
for img_id in X_train + X_val:
    convert_and_resize(img_id, OUTPUT_DIR)


C:\Users\bielp\AppData\Local\Temp\ipykernel_5816\3624342911.py:17: RuntimeWarning: invalid value encountered in divide
  norm = ((arr - p1) / (p99 - p1)) * 255
C:\Users\bielp\AppData\Local\Temp\ipykernel_5816\3624342911.py:18: RuntimeWarning: invalid value encountered in cast
  norm = norm.astype(np.uint8)


## Classificació

### Model 1:

### Model 2:

### Model 3:

### Model 4:

## Detecció

## Innovació